In [1]:
import numpy as np
import pandas as pd

In [2]:
from source import ElasticResponse,read_query
from generator.search import templatize
from generator.utils import load_spacy_model
nlp = load_spacy_model()

In [27]:
er = ElasticResponse()
sql_query = read_query("backup","policy_complaint","retention_status")
df = er.get_response(sql_query,{})

In [29]:
df.sort_values('Count')

,Host Name,STATE,Count
6,RLTIA00008,COMPLIANT,1054
0,MRS-IS-00021,COMPLIANT,1148
2,MRS-IS-00023,COMPLIANT,1176
4,RLTIA00007,COMPLIANT,1209
3,MRS-IS-00023,WARNING,1876
5,RLTIA00007,WARNING,2511
7,RLTIA00008,WARNING,2511
1,MRS-IS-00021,WARNING,2716


In [37]:
text = """You can view summary information about policy status per host.The summary information provides the Host Name, STATE and the total count for each server. The Host RLTIA00007 is most complaint among all other servers. """

In [38]:
text = nlp(text)
nugget = templatize(text, {'_by': ['Host Name','STATE'],'_c': ['Count|sum'],"_sort": ["STATE","-Count"]} , df)
nugget

{% set fh_args = {"_by": ["Host Name", "STATE"], "_sort": ["STATE", "-Count"]}  %}
{% set df = U.gfilter(orgdf, fh_args.copy()) %}
{% set fh_args = U.sanitize_fh_args(fh_args, orgdf) %}
{# Do not edit above this line. #}
You can view summary information about policy status per {{ G.singular(fh_args['_by'][0]).lower() }}.The summary information provides the {{ df.columns[0] }}, {{ df.columns[1] }} and the total count for each server. The Host {{ df["Host Name"].iloc[-2] }} is most complaint among all other servers. 

In [15]:
type(nlp)

spacy.lang.en.English

In [ ]:
nugget.render(df)

In [14]:
df

,Host Name,Days
0,MRS-IS-00021,10.0
1,MRS-IS-00023,3.0
2,RLTIA00007,9.0
3,RLTIA00008,10.0


In [10]:
nugget.variables

{Host Name: {{ df.columns[0] }},
 days: {{ G.plural(fh_args['_sort'][0]).lower() }},
 3: {{ df["Days"].iloc[-1] }}}